Импортируем необходимые библиотеки

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

Подключаемся к базе данных telecomm_csi.db

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

Пишем необходимый запрос

In [3]:
query = """
--выводим необходимые поля в заданном порядке
SELECT u.user_id,
       u.lt_day,
--условие для нового или старого клиента
CASE
    WHEN u.lt_day <= 365 THEN 'Новый'
    ELSE 'Старый'
END AS is_new,
       u.age,
--условие для изменения 1 и 0 на Женщину и Мужчину, и сразу решаем проблему с пропусками, незначительно увеличив количество Мужчин
CASE
    WHEN u.gender_segment = 1 THEN 'Женщина'
    ELSE 'Мужчина'
END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       aseg.title AS age_segment,
       tseg.title AS traffic_segment,
       lseg.title AS lifetime_segment,
       u.nps_score,
--условия по последнему полю
CASE
    WHEN u.nps_score >= 9 THEN 'promoters'
    WHEN u.nps_score <= 6 THEN 'detractors'
    ELSE 'passives'
END AS nps_group
--присоединяем таблицу location
FROM user AS u
LEFT JOIN location AS l ON u.location_id=l.location_id
LEFT JOIN age_segment AS aseg ON u.age_gr_id=aseg.age_gr_id
LEFT JOIN traffic_segment AS tseg ON u.tr_gr_id=tseg.tr_gr_id
LEFT JOIN lifetime_segment AS lseg ON u.lt_gr_id=lseg.lt_gr_id
"""

Выводим первые строки

In [4]:
df = pd.read_sql(query, engine)
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [6]:
df['user_id'].duplicated().sum()

0

Пропуски остались только в колонке возраст. Нашёл интересную странность в данных. В колонке возрастных групп - группа 6 - это возраст в диапазоне 55-64 лет, группа 7 - в диапазоне 66-и более. То есть для 65 лет нет группы!!! Ещё не понятно, куда будут относиться пограничные значения, например в колонке объем трафика значение 0 можно отнести и к 1 группе и ко 2!!! И то же самое в колонке количества месяцев жизни.

In [7]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

Ссылка на презентацию:

https://public.tableau.com/app/profile/antony.besh/viz/BeshNPS/NPS_8